### Grid Visualisation

####  *AUTHOR:* Ehsan Farahbakhsh
####  *CONTACT:* e.farahbakhsh@sydney.edu.au
####  *DATE last modified:* 26/12/2025

This workflow can be used to plot and create animations for any set of reconstructed grids using pyGPlates and GPlately.

In [ ]:
from ipywidgets import interact
import os

import cartopy.crs as ccrs
import cmcrameri.cm as ccm
import gplately
from gplately import PlateModelManager, PlateReconstruction, PlotTopologies
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import matplotlib.pyplot as plt

from plot_grid import *

In [ ]:
# Timespan for analysis
temporal_resolution = 1
time_min = 0
time_max = 230
time_steps = range(time_min, time_max + temporal_resolution, temporal_resolution)

plate_model_dir = "./plate_model"

grid_dir = "./SedimentThickness"
grid_name = "sediment_thickness"
map_dir = "./SedimentThickness_Maps"
# "Seafloor Age (Myr)", "Carbonate Thickness (m)", "Crustal Carbon Density (t m$^{-2}$)", "Sediment Thickness (m)", "Seafloor Spreading Rate (km/Myr)"
cb_label = "Sediment Thickness (m)"

anchor_plate_id = 705705
n_jobs = 4

In [ ]:
# Manually enter the path to the plate motion files
rotation_model = [
    plate_model_dir+"/CombinedRotation.rot",
]

topology_features = [
    plate_model_dir+"/Feature_Geometries.gpml",
    plate_model_dir+"/Iran-Afghan_Topological Points.gpml",
    plate_model_dir+"/Iran-Afghan-Turan_Topological Line.gpml",
    plate_model_dir+"/Plate_Boundaries.gpml",
    plate_model_dir+"/Sabzevar-Sistan_Topological Line.gpml",
    plate_model_dir+"/SCI_Subduction_points.gpml",
    plate_model_dir+"/SCI_Subduction_Topological_lines.gpml",
    plate_model_dir+"/Topological_lines.gpml",
    plate_model_dir+"/Topological_Points.gpml",
]

static_polygons = plate_model_dir+"/Global_EarthByte_GPlates_PresentDay_StaticPlatePolygons.shp"
coastlines = plate_model_dir+"/Global_coastlines_low_res.shp"
continents = plate_model_dir+"/Global_EarthByte_GPlates_PresentDay_ContinentalPolygons.gpml"
COBs = plate_model_dir+"/Global_EarthByte_GeeK07_COB_Terranes_edited.gpml"

# OR use PMM

# pmm = PlateModelManager()
# pm = pmm.get_model("Muller2025")

# rotation_model = pm.get_rotation_model()
# topology_features = pm.get_topologies()

# static_polygons = pm.get_static_polygons()
# coastlines = pm.get_coastlines()
# continents = pm.get_continental_polygons()
# COBs = pm.get_COBs()

plate_reconstruction = PlateReconstruction(
    rotation_model=rotation_model,
    topology_features=topology_features,
    static_polygons=static_polygons,
    anchor_plate_id=anchor_plate_id,
)

gplot = PlotTopologies(
    plate_reconstruction=plate_reconstruction,
    coastlines=coastlines,
    continents=continents,
    COBs=COBs,
    anchor_plate_id=anchor_plate_id,
)

projection = ccrs.Mollweide(central_longitude=60)
# projection = ccrs.Robinson(central_longitude=0)

### Single Time Step

In [ ]:
gplot.time = 0

grd_file =  f"./{grid_dir}/{grid_name}_0Ma.nc"
grd = gplately.grids.read_netcdf_grid(grd_file)
# grd = grd[::-1, :]

fig = plt.figure(figsize=(16, 12))
ax = plt.axes(
    projection=projection,
    facecolor=(plt.cm.colors.to_rgba("darkgray", alpha=0.5)),
)
ax.set_global()

im = gplot.plot_grid(ax, grd.data, cmap=ccm.lapaz_r, alpha=0.7, zorder=1)
# im = gplot.plot_grid(ax, grd.data, cmap=ccm.batlow_r, alpha=0.7, zorder=1)

gplot.plot_coastlines(ax, facecolor="darkgray", edgecolor="none", zorder=2)
gplot.plot_plate_motion_vectors(ax, spacingX=10, spacingY=10, normalise=True, alpha=0.1, zorder=3)
gplot.plot_ridges(ax, color="dimgray", linewidth=1.5, zorder=4)
gplot.plot_transforms(ax, color="dimgray", linewidth=1.5, zorder=4)
gplot.plot_trenches(ax, color="black", zorder=5)
gplot.plot_subduction_teeth(ax, spacing=0.05, color="black", zorder=6)

# Mollweide
gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, x_inline=False, linewidth=1, color="gray", alpha=0.3, linestyle="--", zorder=7)

# Robinson
# gl = ax.gridlines(
#     crs=ccrs.PlateCarree(),
#     # Only x labels on bottom, y labels on left:
#     draw_labels={"bottom": "x", "left": "y"},
#     x_inline=False,
#     linewidth=1,
#     color="gray",
#     alpha=0.3,
#     linestyle="--",
#     zorder=7,
# )

# Mollweide
gl.top_labels=False
gl.bottom_labels=False
# gl.right_labels=False
# gl.left_labels=False

gl.xlabel_style = {"size": 16}
gl.ylabel_style = {"size": 16}

# Mollweide
ax.text(0.49,-0.03, "60°E", transform=ax.transAxes, fontsize=16)
ax.text(0.46,-0.03, "0°", transform=ax.transAxes, fontsize=16)
ax.text(0.40,-0.025, "60°W", transform=ax.transAxes, fontsize=16)

# Robinson
# ax.text(0.39,-0.03, "60°E", transform=ax.transAxes, fontsize=16)
# ax.text(0.49,-0.03, "0°", transform=ax.transAxes, fontsize=16)
# ax.text(0.56,-0.03, "60°W", transform=ax.transAxes, fontsize=16)

cb = fig.colorbar(im, orientation="horizontal", shrink=0.4, pad=0.06, extend="max")
cb.set_label(cb_label, fontsize=16, labelpad=10)
# cb.set_ticks([0, 50, 100, 150, 200])
cb.ax.tick_params(labelsize=16)

# Dummy handle to trigger custom handler
trench_handle = Line2D([], [], color="black")

# Add custom handles
custom_handles = [
    Patch(facecolor="darkgray", edgecolor="none"),
    trench_handle,
    Line2D([0], [0], color="dimgray", lw=2),
]
custom_labels = [
    "Continental Crust",
    "Trench Lines",
    "Mid-Ocean Ridges/\nTransform Faults",
]

# Draw legend
ax.legend(custom_handles, custom_labels, fontsize=16, loc="lower left", bbox_to_anchor=(0, -0.25),
          handler_map={trench_handle: HandlerTrenchLine()})

plt.show()

### Interactive Visualisation

In [ ]:
overall_min, overall_max = find_min_max(grid_dir)
print(f"Overall minimum: {overall_min}\nOverall maximum: {overall_max}")

# p1, p99 = find_percentiles(grid_dir)
p1, p99 = find_percentiles_dask(grid_dir) # memory efficient
print(f"Percentile 1: {p1}\nPercentile 99: {p99}")

In [ ]:
@interact
def show_map(time=time_steps):
    gplot.time = time
    
    grd_filename =  f"{grid_name}_{time}Ma.nc"
    grd_file = os.path.join(grid_dir, grd_filename)
    
    grd = gplately.grids.read_netcdf_grid(grd_file)
    # grd = grd[::-1, :]
    
    fig = plt.figure(figsize=(16, 12))
    ax = plt.axes(
        projection=projection,
        facecolor=(plt.cm.colors.to_rgba("darkgray", alpha=0.5)),
    )
    ax.set_global()

    im = gplot.plot_grid(ax, grd.data, cmap=ccm.lapaz_r, vmin=0, vmax=p99, alpha=0.7, zorder=1)

    gplot.plot_coastlines(ax, facecolor="darkgray", edgecolor="none", zorder=2)
    gplot.plot_plate_motion_vectors(ax, spacingX=10, spacingY=10, normalise=True, alpha=0.1, zorder=3)
    gplot.plot_ridges(ax, color="dimgray", linewidth=1.5, zorder=4)
    gplot.plot_transforms(ax, color="dimgray", linewidth=1.5, zorder=4)
    gplot.plot_trenches(ax, color="black", zorder=5)
    gplot.plot_subduction_teeth(ax, spacing=0.05, color="black", zorder=6)
    
    # Mollweide
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True, x_inline=False, linewidth=1, color="gray", alpha=0.3, linestyle="--", zorder=7)
    
    # Robinson
#     gl = ax.gridlines(
#         crs=ccrs.PlateCarree(),
#         # Only x labels on bottom, y labels on left:
#         draw_labels={"bottom": "x", "left": "y"},
#         x_inline=False,
#         linewidth=1,
#         color="gray",
#         alpha=0.3,
#         linestyle="--",
#         zorder=7,
#     )
    
    # Mollweide
    gl.top_labels=False
    gl.bottom_labels=False
#     gl.right_labels=False
#     gl.left_labels=False
    
    gl.xlabel_style = {"size": 16}
    gl.ylabel_style = {"size": 16}
    
    # Mollweide
    ax.text(0.49,-0.03, "60°E", transform=ax.transAxes, fontsize=16)
    ax.text(0.46,-0.03, "0°", transform=ax.transAxes, fontsize=16)
    ax.text(0.40,-0.025, "60°W", transform=ax.transAxes, fontsize=16)
    
    # Robinson
    # ax.text(0.39,-0.03, "60°E", transform=ax.transAxes, fontsize=16)
    # ax.text(0.49,-0.03, "0°", transform=ax.transAxes, fontsize=16)
    # ax.text(0.56,-0.03, "60°W", transform=ax.transAxes, fontsize=16)
        
    cb = fig.colorbar(im, orientation="horizontal", shrink=0.4, pad=0.06, extend="max")
    cb.set_label(cb_label, fontsize=16, labelpad=10)
#     cb.set_ticks([0, 5000, 10000, 15000])
    cb.ax.tick_params(labelsize=16)
    
    # Dummy handle to trigger custom handler
    trench_handle = Line2D([], [], color="black")

    # Add custom handles
    custom_handles = [
        Patch(facecolor="darkgray", edgecolor="none"),
        trench_handle,
        Line2D([0], [0], color="dimgray", lw=2),
    ]
    custom_labels = [
        "Continental Crust",
        "Trench Lines",
        "Mid-Ocean Ridges/\nTransform Faults",
    ]

    # Draw legend
    ax.legend(custom_handles, custom_labels, fontsize=16, loc="lower left", bbox_to_anchor=(0, -0.25),
              handler_map={trench_handle: HandlerTrenchLine()})

    ax.set_title(f"{time} Ma", fontsize=25, y=1.04)
        
    plt.show()

In [ ]:
if os.path.exists(map_dir):
    print(f"Maps are located in {map_dir}")
else:
    os.makedirs(map_dir, exist_ok=True)
       
    generate_grd_maps(
        rotation_model=rotation_model,
        topology_features=topology_features,
        static_polygons=static_polygons,
        coastlines=coastlines,
        continents=continents,
        COBs=COBs,
        grd_dir=grid_dir,
        grd_filename=grid_name,
        cb_label=cb_label,
        vmin=0,
        vmax=p99,
        projection=projection,
        time_steps=time_steps,
        output_dir=map_dir,
        reverse_lat=False,
        anchor_plate_id=anchor_plate_id,
        n_jobs=n_jobs,
    )
    
    output_filenames = [
        os.path.join(map_dir, f"{grid_name}_map_{t:0.0f}Ma.png")
        for t in time_steps
    ]
    
    output_filename = os.path.join(map_dir, f"{grid_name}_animation.mp4")
    create_animation(
        image_filenames=output_filenames[::-1],
        output_filename=output_filename,
        fps=10,
        bitrate="5000k",
    )